# 🔍 Google AI Studio File Search Tool
Search inside files without extension in your "/Google AI Studio" drive folder

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import re
from pathlib import Path
from IPython.display import HTML, display

# Configuration
DRIVE_FOLDER = '/content/drive/My Drive/Google AI Studio'
ENCODING = 'utf-8'

print("✅ Google Drive mounted!")

In [ ]:
def find_files_without_extension(folder_path):
    """Find all files without extension in the specified folder and subfolders"""
    files_without_ext = []

    if not os.path.exists(folder_path):
        print(f"❌ Folder not found: {folder_path}")
        return files_without_ext

    try:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Check if file has no extension (no dot in filename, or dot is at the start)
                if '.' not in file or file.startswith('.'):
                    full_path = os.path.join(root, file)
                    files_without_ext.append(full_path)
    except PermissionError as e:
        print(f"⚠️  Permission denied: {e}")

    return files_without_ext

# Find files without extension
print("📂 Scanning for files without extension...")
files_without_ext = find_files_without_extension(DRIVE_FOLDER)

if files_without_ext:
    print(f"✅ Found {len(files_without_ext)} files without extension:\n")
    for file in files_without_ext[:15]:
        print(f"  • {file}")
    if len(files_without_ext) > 15:
        print(f"  ... and {len(files_without_ext) - 15} more")
else:
    print("📭 No files without extension found")

print(f"\nTotal files found: {len(files_without_ext)}")

In [ ]:
def search_in_files(files, search_term, case_sensitive=False):
    """Search for a term in the content of files"""
    results = []
    flags = 0 if case_sensitive else re.IGNORECASE

    for file_path in files:
        try:
            with open(file_path, 'r', encoding=ENCODING, errors='ignore') as f:
                content = f.read()
                matches = re.finditer(search_term, content, flags)
                matches_list = list(matches)

                if matches_list:
                    # Get context around matches
                    lines = content.split('\n')
                    for match in matches_list:
                        # Find which line the match is on
                        pos = match.start()
                        line_num = content[:pos].count('\n') + 1
                        
                        # Get line content
                        line_content = lines[line_num - 1] if line_num <= len(lines) else ""
                        
                        results.append({
                            'file': file_path,
                            'line': line_num,
                            'match': match.group(),
                            'line_content': line_content,
                            'position': pos
                        })
        except Exception as e:
            print(f"⚠️  Error reading {file_path}: {e}")

    return results

def display_results_html(results, limit=30):
    """Display search results in HTML format"""
    if not results:
        display(HTML("<p><strong>📭 No matches found</strong></p>"))
        return

    html = f"<p><strong>✅ Found {len(results)} matches</strong></p>"
    html += "<div style='border: 1px solid #ddd; padding: 10px; border-radius: 5px;'>"

    for i, result in enumerate(results[:limit], 1):
        file_name = result['file'].replace('/content/drive/My Drive/', '')
        html += f"<div style='margin-bottom: 15px; padding-bottom: 10px; border-bottom: 1px solid #eee;'>"
        html += f"<p><strong>{i}. {file_name}</strong></p>"
        html += f"<p style='color: #666; font-size: 12px;'>Line {result['line']}</p>"
        html += f"<p style='background-color: #f5f5f5; padding: 5px; border-radius: 3px; word-break: break-all;'>"
        html += f"<code><strong style='color: #d9534f;'>{result['match']}</strong></code></p>"
        
        if result['line_content']:
            context = result['line_content'][:100]
            if len(result['line_content']) > 100:
                context += "..."
            html += f"<p style='color: #999; font-size: 12px;'><em>{context}</em></p>"
        html += "</div>"

    if len(results) > limit:
        html += f"<p><em>... and {len(results) - limit} more matches</em></p>"

    html += "</div>"
    display(HTML(html))

print("🔍 Search functions ready!")

In [ ]:
# Example search - modify these values to search
search_term = "TODO"  # Change this to your search term
case_sensitive = False  # Set to True for case-sensitive search

print(f"🔍 Searching for '{search_term}'...\n")
results = search_in_files(files_without_ext, search_term, case_sensitive)

print(f"Found {len(results)} matches in {len(files_without_ext)} files\n")
display_results_html(results, limit=30)

In [ ]:
# Advanced search example - search with regex
search_term = r"error.*line"  # Regex pattern
case_sensitive = False

print(f"🔍 Searching with regex: '{search_term}'...\n")
results = search_in_files(files_without_ext, search_term, case_sensitive)

print(f"Found {len(results)} matches\n")
display_results_html(results, limit=30)